In [79]:
import requests
import os
import math
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import random

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.manifold import TSNE

from api import PEXELS_API_KEY

In [80]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

class Config:
    dataset_root = "dataset"
    train_ratio = 0.8  
    batch_size = 16
    num_epochs = 10
    lr = 1e-3
    momentum = 0.9
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

cfg = Config()
print(f"Используемое устройство: {cfg.device}")

Используемое устройство: cpu


In [65]:
os.makedirs("dataset/xmas", exist_ok=True)
os.makedirs("dataset/not_xmas", exist_ok=True)

NUM_IMAGES_TOTAL = 200
PER_PAGE = 80

XMAS_QUERIES = [
    "Christmas Tree",
    "Christmas Lights",
    "Christmas Gifts",
    "Snowy Christmas",
    "Santa Claus",
    "Christmas Decoration"
]

NOT_XMAS_QUERIES = [
    "Summer Beach",
    "Urban Landscape",
    "Mountain Hiking",
    "City Skyline",
    "Desert Dunes",
    "Forest Trail"
]

IMAGES_PER_QUERY_XMAS = math.ceil(NUM_IMAGES_TOTAL / len(XMAS_QUERIES))
IMAGES_PER_QUERY_NOT_XMAS = math.ceil(NUM_IMAGES_TOTAL / len(NOT_XMAS_QUERIES))

In [66]:
def fetch_images_from_api(query, page):
    headers = {
        "Authorization": PEXELS_API_KEY
    }
    url = f"https://api.pexels.com/v1/search?query={query}&per_page={PER_PAGE}&page={page}"
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"API Error: {response.status_code}, {response.text}")
        return []
    
    return response.json().get("photos", [])

In [67]:
def save_image(photo, folder, query, index):
    img_url = photo["src"]["large"]
    try:
        img_data = requests.get(img_url).content
        file_path = os.path.join(folder, f"{query.replace(' ', '_')}_{index + 1}.jpg")
        with open(file_path, "wb") as handler:
            handler.write(img_data)
    except Exception as e:
        print(f"Download Error: {e}")

In [68]:
def download_images(query, folder, num_images):
    total_pages = math.ceil(num_images / PER_PAGE)
    downloaded = 0
    
    for page in range(1, total_pages + 1):
        photos = fetch_images_from_api(query, page)
        
        if not photos:
            print("No more images available for this query.")
            break
        
        for idx, photo in enumerate(photos):
            save_image(photo, folder, query, downloaded)
            downloaded += 1
            
            if downloaded >= num_images:
                print(f"Download complete: {downloaded}/{num_images} images downloaded for query '{query}' in folder '{folder}'")
                return
    
    print(f"Download complete: {downloaded}/{num_images} images downloaded for query '{query}' in folder '{folder}'")

In [69]:
for query in XMAS_QUERIES:
    download_images(query, "dataset/xmas", num_images=IMAGES_PER_QUERY_XMAS)

for query in NOT_XMAS_QUERIES:
    download_images(query, "dataset/not_xmas", num_images=IMAGES_PER_QUERY_NOT_XMAS)

Download complete: 34/34 images downloaded for query 'Christmas Tree' in folder 'dataset/xmas'
Download complete: 34/34 images downloaded for query 'Christmas Lights' in folder 'dataset/xmas'
Download complete: 34/34 images downloaded for query 'Christmas Gifts' in folder 'dataset/xmas'
Download complete: 34/34 images downloaded for query 'Snowy Christmas' in folder 'dataset/xmas'
Download complete: 34/34 images downloaded for query 'Santa Claus' in folder 'dataset/xmas'
Download complete: 34/34 images downloaded for query 'Christmas Decoration' in folder 'dataset/xmas'
Download complete: 29/29 images downloaded for query 'Summer Beach' in folder 'dataset/not_xmas'
Download complete: 29/29 images downloaded for query 'Tropical Island' in folder 'dataset/not_xmas'
Download complete: 29/29 images downloaded for query 'Urban Landscape' in folder 'dataset/not_xmas'
Download complete: 29/29 images downloaded for query 'Mountain Hiking' in folder 'dataset/not_xmas'
Download complete: 29/29 i